In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras import Model
import tensorflow as tf
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, SpatialDropout1D
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer

from keras import callbacks
from keras.callbacks import CSVLogger

import os

#traindata = pd.read_csv('preprocessed_train_multiclass(a)_new.csv', header=None)
#testdata = pd.read_csv('preprocessed_test_multiclass(a)_new.csv', header=None)
X=pd.read_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli121.csv", header=None )
T=pd.read_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli11.csv", header=None )
C=pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli1_label.csv",header=None)
Y=pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli2_label.csv",header=None)
'''X = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain_1.csv", header=None)
Y = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\train_label.csv", header=None)

T = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\pva1\pva_1.csv", header=None)
C = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\pva1\pva_label.csv", header=None)'''

X1 = X.iloc[0:,0:76]# 0 to 73 columns of data frame with all rows
T1 = T.iloc[0:,0:76]# 

scaler = Normalizer().fit(X1)
trainX = scaler.transform(X1)


scaler = Normalizer().fit(T1)
testT = scaler.transform(T1)

y_train1 = np.array(Y)
y_test1 = np.array(C)

y_train = to_categorical(y_train1)
y_test = to_categorical(y_test1)

# reshape input to be [samples, time steps, features]
X_train = trainX
#X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
#X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))
X_test = testT

# Add a channels dimension
X_train = X_train[..., tf.newaxis].astype("float64")
X_test = X_test[..., tf.newaxis].astype("float64")
batch_size = 50



train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train1)).shuffle(1000000).batch(50)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test1)).batch(50)


class MyModel(Model):
    #
    def __init__(self):
        #
        super(MyModel, self).__init__()
        self.lstm1=LSTM(50,input_shape=(1, 76), return_sequences= True )
        self.drop1=Dropout(0.01)
        self.lstm2=LSTM(50,return_sequences=True)
        self.drop2=Dropout(0.01)
        self.lstm3=LSTM(50, return_sequences=False)
        self.drop3=Dropout(0.01)
        self.d1=Dense(14)
        self.activation1=Activation('softmax')

    def call(self, x):
        x = self.lstm1(x)
        x = self.drop1(x)
        x = self.lstm2(x)
        x = self.drop2(x)
        x = self.lstm3(x)
        x = self.drop3(x)
        x = self.d1(x)
        return self.activation1(x)

# Create an instance of the model
model = MyModel()

loss_object = tf.keras.losses.CategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

'''
model.add(LSTM(50,input_shape=(1, 76), return_sequences= True ))  
model.add(Dropout(0.01))
model.add(LSTM(50,return_sequences=True)) 
model.add(Dropout(0.01))
#model.add(LSTM(40,return_sequences=True))   
#model.add(Dropout(0.01))
#model.add(LSTM(40, return_sequences=True))  
#model.add(Dropout(0.01))
#model.add(LSTM(40, return_sequences=True)) 
#model.add(Dropout(0.01))
#model.add(LSTM(80, return_sequences=True)) 
#model.add(Dropout(0.01))
model.add(LSTM(50, return_sequences=False)) 
model.add(Dropout(0.01))
model.add(Dense(14))# the no. of output classes
model.add(Activation('softmax'))


model.summary()
'''


@tf.function
def train_step(traind, labels):
    
    with tf.GradientTape() as tape:
        #
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
        predictions = model(traind, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
    )


Epoch 1, Loss: 0.3826783299446106, Accuracy: 88.6814193725586, Test Loss: 0.23311454057693481, Test Accuracy: 92.4338150024414
Epoch 2, Loss: 0.2317880243062973, Accuracy: 92.44519805908203, Test Loss: 0.20232480764389038, Test Accuracy: 93.585693359375
Epoch 3, Loss: 0.19117125868797302, Accuracy: 94.28759002685547, Test Loss: 0.19409720599651337, Test Accuracy: 93.17318725585938
Epoch 4, Loss: 0.16969864070415497, Accuracy: 94.92601013183594, Test Loss: 0.1666862666606903, Test Accuracy: 95.06137084960938
Epoch 5, Loss: 0.15610621869564056, Accuracy: 95.42857360839844, Test Loss: 0.14713887870311737, Test Accuracy: 95.7340087890625


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import OneClassSVM
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
df=pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1.csv")
df2=df.drop_duplicates( keep=False)
print ( df.shape)
print ( df2.shape)

D:\ana\envs\tensorflowb\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1222654, 80)
(1174778, 80)


In [3]:
df2.drop('Protocol',axis=1,inplace=True)
df2.drop('Dst Port',axis=1,inplace=True)
df2.drop('Timestamp',axis=1,inplace=True)
print ( df.shape)
print ( df2.shape)

D:\ana\envs\tensorflowb\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(1222654, 80)
(1174778, 77)


In [4]:
print ( df2.shape )

for index in df2.columns:
    df2[index]=df2[index].fillna(0)
    df2[index]=df2[index].replace(np.Infinity,1)

#X.drop ( columus=[77],axis=1,inplace=True)
#X.to_csv(r"E:\学习心得\data\cic-ids2018\02-03\pva1\pva_1.csv", header=None,index=0)
print (df2.shape)

(1174778, 77)


C:\Users\44742\AppData\Local\Temp/ipykernel_9564/4146149062.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[index]=df2[index].fillna(0)
C:\Users\44742\AppData\Local\Temp/ipykernel_9564/4146149062.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[index]=df2[index].replace(np.Infinity,1)


(1174778, 77)


In [5]:
df2.to_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2.csv", header=None,index=0)

In [6]:
pt1=df2.groupby('Label').apply(pd.DataFrame.sample, frac=0.5).reset_index(level='Label', drop=True)
pt2=df2.drop(pt1.index)

In [8]:
pt1.to_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli1.csv", header=None,index=0 )
pt2.to_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli2.csv", header=None,index=0 )
labelt=pt1.Label.copy()
print ( labelt.value_counts())
arrs=labelt.values
d1=np.array(arrs)
values = array(d1)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
data3 = pd.DataFrame(integer_encoded) # header:原第一行的索引，index:原第一列的索引
data3.to_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli1_labeln.csv",header=None,index=0)
# # binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
data4 = pd.DataFrame(onehot_encoded) # header:原第一行的索引，index:原第一列的索引
data4.to_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli1_label.csv",header=None,index=0)

Benign                      450202
DDOS attack-HOIC             50045
DoS attacks-Hulk             33242
Bot                          20944
Infilteration                12498
SSH-Bruteforce               10042
FTP-BruteForce                4242
DoS attacks-GoldenEye         3050
DoS attacks-SlowHTTPTest      2156
DoS attacks-Slowloris          774
DDOS attack-LOIC-UDP           127
Brute Force -Web                45
Brute Force -XSS                17
SQL Injection                    6
Name: Label, dtype: int64
['Benign' 'Benign' 'Benign' ... 'SSH-Bruteforce' 'SSH-Bruteforce'
 'SSH-Bruteforce']
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [9]:
labelt=pt2.Label.copy()
print ( labelt.value_counts())
arrs=labelt.values
d1=np.array(arrs)
values = array(d1)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
data3 = pd.DataFrame(integer_encoded) # header:原第一行的索引，index:原第一列的索引
data3.to_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli2_labeln.csv",header=None,index=0)
# # binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
data4 = pd.DataFrame(onehot_encoded) # header:原第一行的索引，index:原第一列的索引
data4.to_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli2_label.csv",header=None,index=0)

Benign                      450203
DDOS attack-HOIC             50045
DoS attacks-Hulk             33242
Bot                          20943
Infilteration                12498
SSH-Bruteforce               10043
FTP-BruteForce                4241
DoS attacks-GoldenEye         3049
DoS attacks-SlowHTTPTest      2155
DoS attacks-Slowloris          773
DDOS attack-LOIC-UDP           127
Brute Force -Web                45
Brute Force -XSS                17
SQL Injection                    7
Name: Label, dtype: int64
['Benign' 'Benign' 'Benign' ... 'Infilteration' 'Infilteration'
 'Infilteration']
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras import Model
df2 = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli1.csv", header=None)


for index in df2.columns:
    df2[index]=df2[index].fillna(0)
    df2[index]=df2[index].replace(np.Infinity,1)
    
df2.to_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli11.csv", header=None,index=0 )
print ( df2.shape )

KeyError: 'Flow Pkts/s'

In [16]:
df2 = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli2.csv", header=None)
for index in df2.columns:
    df2[index]=df2[index].fillna(0)
    df2[index]=df2[index].replace(np.Infinity,1)
    
df2.to_csv (r"E:\学习心得\data\cic-ids2018\02-03\ptrain1\ptrain2_sli121.csv", header=None,index=0 )
print ( df2.shape )

(587388, 77)


In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.6.0
